# Generate Graphs for 5-letter words
where an edge represents a one letter difference between two wordsm

In [1]:
from pathlib import Path
from string import ascii_lowercase as lowercase

import networkx as nx
import pandas as pd

from ipyforcegraph.forcegraph import DataFrameSource, ForceGraph, ForceGraph3D
from ipyforcegraph.behaviors import NodeLabels

In [2]:
def generate_word_graph(sourcepath: (str, Path) = "words.txt") -> nx.Graph:
    if isinstance(sourcepath, str):
        sourcepath = Path(sourcepath)

    words = {
        word
        for word in sourcepath.read_text().strip().split("\n")
    }

    G = nx.Graph(name=sourcepath.name)
    lookup = {c: lowercase.index(c) for c in lowercase}

    def edit_distance_one(word):
        for i in range(len(word)):
            left, c, right = word[0:i], word[i], word[i + 1 :]
            j = lookup[c]  # lowercase.index(c)
            for cc in lowercase[j + 1 :]:
                yield left + cc + right

    candgen = (
        (word, cand)
        for word in sorted(words)
        for cand in edit_distance_one(word)
        if cand in words
    )
    G.add_nodes_from(words)
    for word, cand in candgen:
        G.add_edge(word, cand)
    return G


def make_forcegraph(GraphClass=ForceGraph, height="800px"):
    G = generate_word_graph()
    fg = GraphClass(
        source=DataFrameSource(
            nodes=pd.DataFrame(data=list(G.nodes), columns=["id"]),
            links=nx.to_pandas_edgelist(G),
            layout=dict(height=height),
        ),
    )
    fg.layout.height = height
    return fg

In [3]:
fg = make_forcegraph()
fg

ForceGraph(layout=Layout(height='800px'), source=DataFrameSource(nodes=(5757, 1), links=(14135, 2)))

In [12]:
fg.behaviors = [NodeLabels()]